Việc phân loại tập dữ liệu và áp dụng một chức năng cho mỗi nhóm, cho dù là tổng hợp hay chuyển đổi, thường là một thành phần quan trọng của quy trình phân tích dữ liệu.Sau khi tải, hợp nhất và chuẩn bị tập dữ liệu, bạn có thể cần tính toán thống kê nhóm hoặc có thể là bảng tổng hợp cho mục đích báo cáo hoặc trực quan hóa. Pandas cung cấp 1 flexible `group by` interface

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### 10.1 GroupBy Mechanics

Hadley Wickham, tác giả của nhiều gói phổ biến cho ngôn ngữ lập trình R, đã đặt ra thuật ngữ `split-apply-merge` để mô tả các hoạt động của `group()`. Trong giai đoạn đầu tiên của quy trình, dữ liệu chứa trong đối tượng **pandas**, dù là Series, DataFrame hay cách khác, được `split()` các nhóm dựa trên một hoặc nhiều khóa mà bạn cung cấp. Việc phân tách được thực hiện trên một trục cụ thể của một đối tượng. Ví dụ: một DataFrame có thể được nhóm trên các hàng (trục = 0) hoặc các cột của nó (trục = 1). Khi điều này được thực hiện, một hàm được áp dụng cho mỗi nhóm, tạo ra một giá trị mới. Cuối cùng, kết quả của tất cả các ứng dụng chức năng đó được `combine()` thành một đối tượng kết quả. Hình thức của đối tượng kết quả thường sẽ phụ thuộc vào những gì đang được thực hiện với dữ liệu ”

In [2]:
from IPython.display import Image
Image(url="./Images/Illustration of a group aggregation.png")

Mỗi nhóm key có thể lấy rất nhiều forms, và các key không có tất cả cùng kiểu loại

1. 1 List hoặc 1 mảng của giá trị rằng có thể cùng 1 chiều với trục đang nhóm

2. Một giá trị cho biết tên cột trong 1 Data frame

3. A dict hoặc 1 series cho bạn Đưa ra sự tương ứng giữa các giá trị trên trục được nhóm và tên nhóm"

4. A hàm có thể thực thi trên 1 axis indexx hoặc label trong index

*Lưu ý rằng ba phương pháp sau là các phím tắt để tạo ra một mảng giá trị được sử dụng để chia nhỏ đối tượng*

**Example**

In [3]:
df = pd.DataFrame({
    'key1': ['a','a','b','b','a'],
    'key2': ['one','two','one','two','one'],
    'data1': np.random.randn(5),
    'dat2': np.random.randn(5)
})

In [4]:
df

,key1,key2,data1,dat2
0,a,one,1.026645,-0.945915
1,a,two,-1.139280,-0.125311
2,b,one,-1.005033,0.792579
3,b,two,-0.962221,-0.657711
4,a,one,0.066520,-1.301189


Giả sử bạn muốn tính giá trị trung bình của cột **data1** bằng cách sử dụng các nhãn từ **key1**. Có một số cách để làm điều này. Một là truy cập data1 và gọi nhóm theo cột (a Series) tại key1:

In [5]:
grouped = df['data1'].groupby(df['key1'])

In [6]:
grouped


biến `grouped ` là một **GroupBy Object**. Nó chưa thực sự tính toán bất cứ điều gì cả, nó chỉ gộp các nhóm lại với nhau thôi 

In [7]:
grouped.mean()

key1
a   -0.015372
b   -0.983627
Name: data1, dtype: float64

Giờ chúng ta sẽ giải thích cái gì đã xảy ra `.mean()`. Điều quan trọng ở đây là a `Series` đã gộp lại theo group `key`, sẽ làm ra 1 new `Series mới` thực hiện gộp các index trong `key1`. Kết quả sẽ là index của `key1` đã có



Nếu thay vì chúng ta truyền nhiều array như 1 list, chúng ta sẽ truyền nhiều tham số như 1 list chúng ta sẽ đc 1 cái khác biệt

In [8]:
df

,key1,key2,data1,dat2
0,a,one,1.026645,-0.945915
1,a,two,-1.139280,-0.125311
2,b,one,-1.005033,0.792579
3,b,two,-0.962221,-0.657711
4,a,one,0.066520,-1.301189


In [9]:

means = df['data1'].groupby([df['key1'],df['key2']]).mean()


In [10]:
means

key1  key2
a     one     0.546582
      two    -1.139280
b     one    -1.005033
      two    -0.962221
Name: data1, dtype: float64

now chúng ta đang gộp dữ liệu với two keys. 

In [11]:
means.unstack()

key2,one,two
key1,,
a,0.546582,-1.139280
b,-1.005033,-0.962221


Trong ví dụ này, các khóa nhóm đều là Chuỗi, mặc dù chúng có thể là bất kỳ mảng nào có độ dài phù hợp:”

In [12]:
states = np.array(['Ohio','California','California','Ohio','Ohio'])

In [13]:
years = np.array([2005,2005,2006,2005,2006])

In [14]:
df['data1'].groupby([states,years]).mean()

California  2005   -1.139280
            2006   -1.005033
Ohio        2005    0.032212
            2006    0.066520
Name: data1, dtype: float64

In [15]:
df

,key1,key2,data1,dat2
0,a,one,1.026645,-0.945915
1,a,two,-1.139280,-0.125311
2,b,one,-1.005033,0.792579
3,b,two,-0.962221,-0.657711
4,a,one,0.066520,-1.301189


Thông tin nhóm thường được tìm thấy trong DataFrame giống với dữ liệu bạn muốn làm việc. Trong trường hợp đó, bạn có thể chuyển tên cột (cho dù đó là chuỗi, số hay các đối tượng Python khác) làm khóa nhóm

In [16]:
df.groupby('key1').mean()

,data1,dat2
key1,,
a,-0.015372,-0.790805
b,-0.983627,0.067434


In [17]:
df

,key1,key2,data1,dat2
0,a,one,1.026645,-0.945915
1,a,two,-1.139280,-0.125311
2,b,one,-1.005033,0.792579
3,b,two,-0.962221,-0.657711
4,a,one,0.066520,-1.301189


In [18]:
df.groupby(['key1','key2']).mean()

data1      dat2
key1 key2                    
a    one   0.546582 -1.123552
     two  -1.139280 -0.125311
b    one  -1.005033  0.792579
     two  -0.962221 -0.657711

Group chỉ áp dụng với dữ liệu số 

`size()` để trả về kích thước tổng pt nó gộp vào 

In [19]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [20]:
df.groupby('key1').size()

key1
a    3
b    2
dtype: int64

### Iterating Over Groups (Lặp các nhóm)
Đối tượng GroupBy hỗ trợ lặp lại, tạo ra một chuỗi `2 tuple` chứa `tên``nhóm` cùng với phần dữ liệu. Hãy xem xét những điều sau: ”

In [21]:
for name,group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1      dat2
0    a  one  1.026645 -0.945915
1    a  two -1.139280 -0.125311
4    a  one  0.066520 -1.301189
b
  key1 key2     data1      dat2
2    b  one -1.005033  0.792579
3    b  two -0.962221 -0.657711


Trong trường hợp của nhiều key, phần tử đầu tiên trong `tuple` sẽ là 1 tuple của key xem ví dụ ở dứoi 

In [22]:
for (k1,k2), group in df.groupby(['key1','key2']):
    print((k1,k2)) # Key of group 
    print(group) # data

('a', 'one')
  key1 key2     data1      dat2
0    a  one  1.026645 -0.945915
4    a  one  0.066520 -1.301189
('a', 'two')
  key1 key2    data1      dat2
1    a  two -1.13928 -0.125311
('b', 'one')
  key1 key2     data1      dat2
2    b  one -1.005033  0.792579
('b', 'two')
  key1 key2     data1      dat2
3    b  two -0.962221 -0.657711


## Selecting a Column or Subset of Columns Chọn một cột hoặc một tập hợp con các cột

indexing a Groupby Object đã tạo từ 1 DataFrame với 1 tên một cột của mảng của cột tên có hiệu ứng subsetting cho 1 kết hợp: 
Nghĩa là mình chỉ group 1 cột thôI

In [23]:
df.groupby('key1')['data1']

In [24]:
df.groupby('key1')[['dat2']] 

In [25]:
df

,key1,key2,data1,dat2
0,a,one,1.026645,-0.945915
1,a,two,-1.139280,-0.125311
2,b,one,-1.005033,0.792579
3,b,two,-0.962221,-0.657711
4,a,one,0.066520,-1.301189


In [26]:
# Gộp 2 key và chỉ tính mean ở dat2 
df.groupby(['key1','key2'])[['dat2']].mean()

dat2
key1 key2          
a    one  -1.123552
     two  -0.125311
b    one   0.792579
     two  -0.657711

Đối tượng được trả về bởi thao tác lập chỉ mục này là DataFrame được nhóm lại nếu một danh sách hoặc mảng được chuyển hoặc một Chuỗi được nhóm nếu chỉ một tên cột được chuyển dưới dạng một đại lượng vô hướng

In [27]:
s_grouped = df.groupby(['key1','key2'])['dat2']

In [28]:
s_grouped

In [29]:
s_grouped.mean()

key1  key2
a     one    -1.123552
      two    -0.125311
b     one     0.792579
      two    -0.657711
Name: dat2, dtype: float64

### Group with Dicts and Series
Nhóm thông tin có thể tồn tại ở dạng khác với mảng.

In [30]:
people = pd.DataFrame(np.random.randn(5,5),
                         columns =['a','b','c','d','e'],
                         index = ['Joe','Steve','Wes','Jim','Travis'])

In [31]:
people

,a,b,c,d,e
Joe,0.768861,-0.537769,1.573604,-0.239059,-0.039600
Steve,0.844477,1.437527,1.153058,0.852134,0.912319
Wes,-0.103566,0.112803,0.578685,1.208644,-1.637798
Jim,-1.630570,-1.958019,1.057622,1.174837,-0.271501
Travis,0.739819,-0.803692,0.598995,2.000389,-1.591378


In [32]:
people.iloc[2:3,[1,2]]= np.nan

In [33]:
people

,a,b,c,d,e
Joe,0.768861,-0.537769,1.573604,-0.239059,-0.039600
Steve,0.844477,1.437527,1.153058,0.852134,0.912319
Wes,-0.103566,NaN,NaN,1.208644,-1.637798
Jim,-1.630570,-1.958019,1.057622,1.174837,-0.271501
Travis,0.739819,-0.803692,0.598995,2.000389,-1.591378


Giả sử tôi có một nhóm tương ứng cho các cột và muốn tổng hợp các cột lại với nhau theo nhóm"

In [34]:
mapping = {
    'a':'red',
    'b':'red',
    'c':'blue',
    'd':'blue',
    'e':'red',
    'f':'orange'
}

bây giờ, bạn có thể xây dựng một mảng từ dit để truyền cho groupby 

In [35]:
by_column = people.groupby(mapping,axis=1)


In [36]:
by_column

In [37]:
by_column.sum()

,blue,red
Joe,1.334545,0.191492
Steve,2.005192,3.194323
Wes,1.208644,-1.741364
Jim,2.232459,-3.860090
Travis,2.599384,-1.655251


In [38]:
by_column = people.groupby(mapping)

In [39]:
by_column.sum()

,a,b,c,d,e


In [40]:
map_series = pd.Series(mapping)

In [41]:
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [42]:
people.groupby(map_series,axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


In [43]:
people

,a,b,c,d,e
Joe,0.768861,-0.537769,1.573604,-0.239059,-0.039600
Steve,0.844477,1.437527,1.153058,0.852134,0.912319
Wes,-0.103566,NaN,NaN,1.208644,-1.637798
Jim,-1.630570,-1.958019,1.057622,1.174837,-0.271501
Travis,0.739819,-0.803692,0.598995,2.000389,-1.591378


### Grouping with Functions 

Sử dụng các hàm Python là một cách chung chung hơn để xác định 
ánh xạ nhóm so với một dict hoặc Series. Bất kỳ hàm nào được truyền dưới dạng khóa nhóm sẽ được gọi một lần cho mỗi giá trị chỉ mục, với các giá trị trả về được sử dụng làm tên nhóm. Cụ thể hơn, hãy xem xét ví dụ DataFrame từ phần trước, có tên của mọi người làm giá trị chỉ mục. Giả sử bạn muốn nhóm theo độ dài của tên; trong khi bạn có thể tính một mảng độ dài chuỗi, đơn giản hơn là chỉ cần chuyển hàm len: ”

In [44]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.965274,-2.495789,2.631226,2.144421,-1.948899
5,0.844477,1.437527,1.153058,0.852134,0.912319
6,0.739819,-0.803692,0.598995,2.000389,-1.591378


### Grouping by Index Levels 

    Một tiện ích cuối cùng cho các tập dữ liệu được lập chỉ mục phân cấp là khả năng tổng hợp bằng cách sử dụng một trong các cấp của chỉ mục trục. Hãy xem một ví dụ: ”


In [45]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                        [1, 3, 5, 1, 3]],
                                        names=['cty', 'tenor'])



In [46]:
columns

MultiIndex([('US', 1),
            ('US', 3),
            ('US', 5),
            ('JP', 1),
            ('JP', 3)],
           names=['cty', 'tenor'])

In [47]:
hier_df = pd.DataFrame(np.random.randn(4,5),
                      columns=columns)


In [48]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      1.941585  1.053087  0.252795  0.541502 -0.892868
1     -2.233153 -0.342503 -0.351389 -0.421170 -0.852638
2     -0.507047  1.127337 -0.187463  1.659741  0.414442
3     -0.192939 -1.162674 -0.204650  0.270025 -0.629297

In [49]:
hier_df.groupby(level='cty',axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


### 10.2 Data Aggregation 

In [51]:
data = {'Function name': 
            ['count',
             'sum',
             'mean',
             'median',
             'std,var',
             'min,max',
             'prod',
             'first,last'],
        'Description': [
            "Number of non-NA values in the group",
            "Sum of non-NA",
            "Mean of non-Na",
            'Median of non-Na',
            "Độ lệch chuẩn và phương sai",
            "Min và max values",
            "Product of non-Na values",
            "first and last non-Na values"
        ]
       }

In [52]:
groupby_methods = pd.DataFrame(data,columns=['Function name','Description'])

In [53]:
groupby_methods

,Function name,Description
0,count,Number of non-NA values in the group
1,sum,Sum of non-NA
2,mean,Mean of non-Na
3,median,Median of non-Na
4,"std,var",Độ lệch chuẩn và phương sai
5,"min,max",Min và max values
6,prod,Product of non-Na values
7,"first,last",first and last non-Na values


In [54]:
df

,key1,key2,data1,dat2
0,a,one,1.026645,-0.945915
1,a,two,-1.139280,-0.125311
2,b,one,-1.005033,0.792579
3,b,two,-0.962221,-0.657711
4,a,one,0.066520,-1.301189


In [55]:
groued = df.groupby('key1')

In [56]:
groued['data1'].quantile(0.9)

key1
a    0.834620
b   -0.966502
Name: data1, dtype: float64

Để sử dụng function của bạn trong aggregation, truywwfn bất kỳ function nào tại araray tới thằng aggrete or `agg()`

In [57]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [58]:
groued.agg(peak_to_peak)

,data1,dat2
key1,,
a,2.165925,1.175878
b,0.042812,1.450290


Bạn có thể thất rằng các methods như `describe()` cũng hoạt động ngay cả khi chúng k gộp lại, 


In [59]:
groued.describe()

data1                                                             \
     count      mean       std       min      25%       50%       75%   
key1                                                                    
a      3.0 -0.015372  1.085282 -1.139280 -0.53638  0.066520  0.546582   
b      2.0 -0.983627  0.030273 -1.005033 -0.99433 -0.983627 -0.972924   

                dat2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.026645   3.0 -0.790805  0.603089 -1.301189 -1.123552 -0.945915   
b    -0.962221   2.0  0.067434  1.025510 -0.657711 -0.295138  0.067434   

                          
           75%       max  
key1                      
a    -0.535613 -0.125311  
b     0.430007  0.792579

## Column-Wise and multiple Function Applicaiton 

Example

In [60]:
tips = pd.read_csv("../examples/tips.csv")

In [61]:
tips

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4
...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3
240,27.18,2.00,Yes,Sat,Dinner,2
241,22.67,2.00,Yes,Sat,Dinner,2
242,17.82,1.75,No,Sat,Dinner,2


In [62]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [63]:
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


. Tuy nhiên, bạn có thể muốn tổng hợp bằng một hàm khác tùy thuộc vào cột hoặc nhiều hàm cùng một lúc.

In [64]:
grouped = tips.groupby(['day','smoker'])

In [65]:
grouped_pct = grouped['tip_pct']

In [66]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

 Nếu khi bạn truyền 1 function hoặc tên của function , bạn sẽ lấy được 1 DataFrame với tên cột lấy từ functions : 

In [71]:
grouped_pct.agg(['mean','std'])

mean       std
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

Dưới đây tôi sẽ truyền vào 1 danh sách cho `aggregation()`to `agg()`

Nếu bạn truyền một danh sách của 1 tuple (name,func),('bả,function. Thì thử phần tử đầu tiên ủa mỗi tuple thường sử dụng như là 1 têm cột 
DataFrame 

In [74]:
grouped_pct.agg([('foo','mean'),('bar','std')])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

Với một DataFrame bạn có nhiều sự lựa chonh,  như bạn có thể chỉh sửa một danh sách của function đê ápp dụng tất cả các c

In [78]:
funcitons = ['count','mean','max']
result= grouped[['tip_pct','total_bill']].agg(funcitons)

In [79]:
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [80]:
ftuples = [('Durchschnitt','mean'),('Abweichung','var')]



In [81]:
grouped[['tip_pct','total_bill']].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

**NOTE**: Nó còn sử dụng chỉ định key cho các agg name fucntion bằng sử dụng `dict`